In [8]:
import os
import sys
import requests

from dotenv import load_dotenv
import s3fs
from fastapi import FastAPI
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel
import uvicorn

PROJECT_PATH = '/home/onyxia/work/llm-open-data-insee/'
sys.path.append(os.path.join(PROJECT_PATH, 'src')) 

from config import RAG_PROMPT_TEMPLATE
from model_building import build_llm_model
from chain_building import load_retriever, build_chain

# Load env variables from .env file
load_dotenv()

# S3 configuration
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

## Query RAG directly

In [5]:
# Import Chroma DB from S3
DB_PATH_S3 = os.path.join(os.environ["S3_BUCKET"], os.environ["DB_KEY_S3"])
DB_PATH_LOCAL = os.path.join(PROJECT_PATH, "data", "chroma_db")
fs.get(DB_PATH_S3, DB_PATH_LOCAL, recursive=True)

# Generate prompt template
prompt = PromptTemplate(input_variables=["context", "question"], template=RAG_PROMPT_TEMPLATE)

# Create a pipeline with tokenizer and LLM
retriever = load_retriever(DB_PATH_LOCAL)
llm = build_llm_model(quantization_config=True, config=True, token=os.environ["HF_TOKEN"])
chain = build_chain(retriever, prompt, llm)

/opt/mamba/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [27]:
question = "Je cherche à connaitre le nombre (et eventuellement les caractéristiques) des véhicules 'primes à la conversion' dans plusieurs départements d'occitanie, en particulier l'aveyron."
response = chain.invoke(question)
response

saving outputs in logs/conversation_logs.json


{'context': [Document(page_content='. (2) «Le Lot, un département rural dynamisé par l’industrie et le tourisme», Insee Analyses Occitanie no132, avril 2023. (3) «La Lozère, des atouts pour résister aux crises», Insee Analyses Occitanie no131, avril 2023. (4) «Les dynamiques démographiques dans les départements d’Occitanie», Insee Dossier Occitanie no17, janvier 2023. (5) «L’Occitanie, première région française pour le nombre de résidences secondaires», Insee Analyses Occitanie no107, juillet 2021. (6) «Un habitant sur deux potentiellement exposé à de fortes chaleurs à répétition dans les prochaines années», Insee Analyses Occitanie no92, février 2020. (7) Résultats du recensement agricole 2020.', metadata={'insee_id': 7344780, 'source': 'https://www.insee.fr/fr/statistiques/7344780', 'start_index': 19535, 'title': 'Le Gers, un département rural sous forte influence toulousaine à l’est'}),
  Document(page_content='. Elle devance légèrement la Nouvelle-Aquitaine. De même, l’Occitanie se

In [37]:
def extract_context_as_dict(response):
    dict_context = {}
    for i, doc in enumerate(response['context']):
        dict_context[i] = doc.metadata
        dict_context[i]['content'] = doc.page_content
    return dict_context

## Query RAG through locally deployed API

In [38]:
CHATBOT_URL = "https://user-avouacr-879366-user.user.lab.sspcloud.fr/chat"

In [39]:
question = "Je cherche à connaitre le nombre (et eventuellement les caractéristiques) des véhicules 'primes à la conversion' dans plusieurs départements d'occitanie, en particulier l'aveyron."
query = {"question": question}
output = requests.post(CHATBOT_URL, json=query)
print(output)

<Response [200]>


In [44]:
output.json()

{'context': {'0': {'insee_id': 7344780,
   'source': 'https://www.insee.fr/fr/statistiques/7344780',
   'start_index': 19535,
   'title': 'Le Gers, un département rural sous forte influence toulousaine à l’est',
   'content': '. (2) «Le Lot, un département rural dynamisé par l’industrie et le tourisme», Insee Analyses Occitanie no132, avril 2023. (3) «La Lozère, des atouts pour résister aux crises», Insee Analyses Occitanie no131, avril 2023. (4) «Les dynamiques démographiques dans les départements d’Occitanie», Insee Dossier Occitanie no17, janvier 2023. (5) «L’Occitanie, première région française pour le nombre de résidences secondaires», Insee Analyses Occitanie no107, juillet 2021. (6) «Un habitant sur deux potentiellement exposé à de fortes chaleurs à répétition dans les prochaines années», Insee Analyses Occitanie no92, février 2020. (7) Résultats du recensement agricole 2020.'},
  '1': {'insee_id': 6522575,
   'source': 'https://www.insee.fr/fr/statistiques/6522575',
   'start_i